# Excluding familiarization trials
Use this notebook to generate a list of excluded game IDs

**Criteria**:
* are correct for fewer than 4 out of 10 familiarization trials (i.e., 30% correct or lower)

## setup

#### Load packages

In [1]:
import os
import sys
import urllib, io

sys.path.append('./analysis_helpers')
from importlib import reload

import numpy as np
import scipy.stats as stats
import pandas as pd

import analysis_helpers as h

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

from tqdm.notebook import tqdm

import  matplotlib
from matplotlib import pylab, mlab, pyplot
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
plt.style.use('seaborn-white')

import seaborn as sns

%matplotlib inline
import scipy.stats
import sklearn.metrics
import random

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

#### options

In [2]:
# display all columns
pd.set_option('display.max_columns', None)

# seaborn plotting themes
sns.set_context('talk')
sns.set_style("whitegrid")

#### set up paths and directories

In [3]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('.')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
## add helpers to python path
if os.path.join(analysis_dir,'utils') not in sys.path:
    sys.path.append(os.path.join(analysis_dir,'utils'))   

def make_dir_if_not_exists(dir_name):   
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return dir_name

## create directories that don't already exist        
result = [make_dir_if_not_exists(x) for x in [results_dir,plot_dir,csv_dir]]

### load human data

In [4]:
from experiment_meta import *
HEM = pd.DataFrame(NEURIPS2021_EXPS) # HEM = "human experiment metadata"
HEM

,study,bucket_name,stim_version,iterationName
0,dominoes_pilot,human-physics-benchmarking-dominoes-pilot,production_1,production_1_testing
1,collision_pilot,human-physics-benchmarking-collision-pilot,production_2,production_2_testing
2,towers_pilot,human-physics-benchmarking-towers-pilot,production_2,production_2_testing
3,linking_pilot,human-physics-benchmarking-linking-pilot,production_2,production_2_testing
4,containment_pilot,human-physics-benchmarking-containment-pilot,production_2,production_2_testing
5,rollingsliding_pilot,human-physics-benchmarking-rollingsliding-pilot,production_2,production_2_testing
6,drop_pilot,human-physics-benchmarking-drop-pilot,production_2,production_2_testing
7,clothiness_pilot,human-physics-benchmarking-clothiness-pilot,production_2,production_2_testing


In [5]:
SCENARIOS = sorted([n.split("_")[0] for n in HEM['study'].unique()])

In [7]:
## get paths to all human response data
data_paths = [os.path.join(csv_dir,'humans',i) for i in os.listdir(os.path.join(csv_dir,'humans'))]
resp_paths = [i for i in data_paths if i.split('/')[-1].split('-')[0]=='familiarization_human_responses']
assert len(resp_paths)==8

In [9]:
## also load all human data into a big dataframe
HD = pd.concat([pd.read_csv(p) for p in resp_paths])
print("Loaded {} lines".format(len(HD)))

Loaded 8030 lines


In [15]:
C_df = HD.groupby('gameID').agg({'correct':['sum','count']})

In [21]:
C_df

correct       ratio
                                              sum count      
gameID                                                       
0021-59322aef-429f-466c-b197-b375d995838e       9    10   0.9
0038-fa81321d-c085-4fcb-b2d3-4ae44aa6e826       8    10   0.8
0051-3828078d-e14d-4b92-8afa-d09b68614808       6    10   0.6
0079-c5d6b258-1168-4049-90e0-f7914f95c0ec       9    10   0.9
0095-611005d8-0573-437d-9023-485d2bb67302       4    10   0.4
...                                           ...   ...   ...
9930-aa52e4be-e5e3-441a-9cb4-f1144d9e233f       3    10   0.3
9933-ddaf8eeb-da07-4846-8465-dbdd503bff92       8    10   0.8
9963-41baa68c-458f-49de-9b53-32286efdcf76       7    10   0.7
9972-65980fc4-2376-4d8b-b66d-d879796fbaad       9    10   0.9
9989-1183e00e-22c2-4441-b896-5cd8e8853704       8    10   0.8

[803 rows x 3 columns]

In [35]:
# get ratio
C_df['ratio'] = C_df[('correct','sum')]/C_df[('correct','count')]

In [37]:
C_df_excluded = C_df[C_df['ratio'] <= .3]

In [38]:
print("We exclude {} games".format(len(C_df_excluded)))

We exclude 9 games


In [45]:
C_df_excluded

correct       ratio
                                              sum count      
gameID                                                       
0720-d5f527dc-d86a-4d88-af8f-b70ac9264fef       3    10   0.3
1685-8963fea0-0d21-454b-8bbe-e9cbc792aa11       3    10   0.3
4917-538725a5-383f-462b-9ab7-43b9473c9dcc       3    10   0.3
7411-987b0a97-8a67-41a3-a3d8-d8f792c35ab5       3    10   0.3
8383-e0582a4e-6498-4d91-bb29-2b6a363cc2e9       3    10   0.3
8731-67e86658-28ff-4cc6-b722-9620e3b3ce43       3    10   0.3
9784-7a67e88b-0416-4b55-8a72-9a0d99038c49       3    10   0.3
9808-e983d3b8-75c3-428a-8182-f57fd645abb1       3    10   0.3
9930-aa52e4be-e5e3-441a-9cb4-f1144d9e233f       3    10   0.3

In [44]:
# Save out to file
pd.DataFrame(C_df_excluded.index).to_csv(os.path.join(csv_dir,"humans/excluded_games.csv"))